In [149]:
class Node:
    def __init__(self, x, y, node_type):
        self.x = x
        self.y = y
        self.node_type = node_type  # 'home', 'food', or 'empty'
        self.neighbors = []  # List of edges to neighboring nodes


In [150]:
def add_homes_and_food_sources(nodes, homes_coords, food_coords):
    for (x, y) in homes_coords:
        nodes[x][y].node_type = 'home'
    for (x, y) in food_coords:
        nodes[x][y].node_type = 'food'


In [151]:
def evaporate_pheromones(edges, evaporation_rate):
    for edge in edges:
        edge.pheromone_level *= (1 - evaporation_rate)


In [152]:
class Edge:
    def __init__(self, node1, node2):
        self.node1 = node1
        self.node2 = node2
        self.pheromone_level = {node1: 1, node2: 1}  # Separate pheromone levels for each direction

    def other_end(self, node):
        return self.node1 if node == self.node2 else self.node2


In [153]:
class Ant:
    def __init__(self, home_node):
        self.current_node = home_node
        self.has_food = False

    
    
    def choose_edge(self):
        if not self.current_node.neighbors:
            return None
        if np.random.rand() < 0.1:  # 10% chance of random exploration
            return np.random.choice(self.current_node.neighbors)
        total_pheromone = sum(edge.pheromone_level[self.current_node] for edge in self.current_node.neighbors)
        probabilities = [edge.pheromone_level[self.current_node] / total_pheromone for edge in self.current_node.neighbors]
        chosen_edge = np.random.choice(self.current_node.neighbors, p=probabilities)
        return chosen_edge
    
    def move_towards(self, target_node):
        angle = math.atan2(target_node.y - self.current_node.y, target_node.x - self.current_node.x)
        self.current_node.x += math.cos(angle)
        self.current_node.y += math.sin(angle)
        # Check if the ant has reached the target node
        if math.hypot(self.current_node.x - target_node.x, self.current_node.y - target_node.y) < 1:
            self.current_node = target_node

    def move(self,ants):
        chosen_edge = self.choose_edge()
        if chosen_edge is None:
            return 
        next_node = chosen_edge.other_end(self.current_node)
        if not any(ant.current_node == next_node for ant in ants):  # Check if the next node is occupied
            self.move_towards(next_node)
            
        self.current_node = chosen_edge.other_end(self.current_node)
        if self.current_node.node_type == 'food':
            self.has_food = True
            self.current_node.food_amount -= 1
            if self.current_node.food_amount == 0:
                self.current_node.node_type ='empty'
        elif self.current_node.node_type == 'home':
            self.has_food = False
        pheromone_deposit = 3 if self.has_food else 1  # Deposit more pheromone when carrying food
        chosen_edge.pheromone_level[self.current_node] += pheromone_deposit


In [154]:
class FoodSource(Node):
    def __init__(self, x, y):
        super().__init__(x, y, 'food')
        self.food_amount = 100
        
class Home(Node):
    def __init__(self, x, y):
        super().__init__(x, y, 'home')

        
        
def create_graph(grid_size, homes_coords, food_coords):
    nodes = [[Node(x, y, 'empty') for y in range(grid_size)] for x in range(grid_size)]
    for x, y in homes_coords:
        nodes[x][y] = Home(x, y)  
    for x, y in food_coords:  
        nodes[x][y] = FoodSource(x, y)
        
    edges = []

    for x in range(grid_size):
        for y in range(grid_size):
            node = nodes[x][y]
            if x > 0:
                left_node = nodes[x - 1][y]
                left_edge = Edge(node, left_node)
                node.neighbors.append(left_edge)
                left_node.neighbors.append(left_edge)
                edges.append(left_edge)
            if y > 0:
                above_node = nodes[x][y - 1]
                above_edge = Edge(node, above_node)
                node.neighbors.append(above_edge)
                above_node.neighbors.append(above_edge)
                edges.append(above_edge)

    return nodes, edges




def create_new_food_source(nodes):
    while True:
        x, y = np.random.randint(len(nodes)), np.random.randint(len(nodes[0]))
        if nodes[x][y].node_type == 'empty':
            nodes[x][y] = FoodSource(x, y)
            break


In [155]:
def evaporate_pheromones(edges, evaporation_rate):
    for edge in edges:
        edge.pheromone_level[edge.node1] *= (1 - evaporation_rate)
        edge.pheromone_level[edge.node2] *= (1 - evaporation_rate)


In [167]:
import pygame
import numpy as np
import math

# ... (other class definitions)

def main():
    pygame.init()
    
    # Screen dimensions
    width, height = 800, 600
    screen = pygame.display.set_mode((width, height))
    
    grid_size = 20
    homes_coords = [(5, 10), (15, 10)]
    food_coords = [(10, 5), (10, 15)]
    nodes, edges = create_graph(grid_size, homes_coords, food_coords)
    

    ant_image = pygame.image.load('ant.webp')
    ant_image = pygame.transform.scale(ant_image, (20, 20))
    
    ants = [Ant(nodes[x][y]) for (x, y) in homes_coords for _ in range(30)]  # 10 ants per home
    
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        for ant in ants:
            ant.move(ants)
        
        evaporate_pheromones(edges, 0.05)  # 1% evaporation rate
        
        # ... (rendering code, to be added in the next step)
        # ... (inside the main() function, replace the comment # ... (rendering code, to be added in the next step) with the following)

        # Clear the screen
        screen.fill((255, 255, 255))
        
        # Draw edges (pheromone trails)
        
        
            
        for edge in edges:
            color = (200, 200, 200)  # Gray color
            pygame.draw.line(screen, color, (edge.node1.x * 20, edge.node1.y * 20), (edge.node2.x * 20, edge.node2.y * 20), 1)
        
        
        # Draw nodes
        for row in nodes:
            for node in row:
                color = (255, 255, 255)  # Default to black
                if node.node_type == 'home':
                    color = (0, 0, 255)  # Blue for homes
                    pygame.draw.circle(screen, color, (node.x * 20, node.y * 20), 5)
                elif node.node_type == 'food' and isinstance(node, FoodSource):
                    
                    color = (255, 0, 0)  # Red for food sources
                    radius = int(math.sqrt(node.food_amount))  # Size based on the amount of food
                    pygame.draw.circle(screen, color, (node.x * 20, node.y * 20), radius)
                
        
        # Draw ants
        for ant in ants:
            color = (0, 255, 0)  # Green for ants
            screen.blit(ant_image, (ant.current_node.x * 20, ant.current_node.y * 20))

            #pygame.draw.circle(screen, color, (ant.current_node.x * 20, ant.current_node.y * 20), 3)

        
        pygame.display.flip()
        pygame.time.Clock().tick(60)  # Limit frame rate to 60 FPS
    
    pygame.quit()


In [170]:
class Ant:
    def __init__(self, home_node):
        self.current_node = home_node
        self.has_food = False

    def sense_environment(self):
        # Check the pheromone levels on neighboring edges or nodes
        pheromone_levels = {edge: edge.pheromone_level[self.current_node] for edge in self.current_node.neighbors}
        return pheromone_levels
    
    def follow_gradient(self, pheromone_levels):
        if not self.current_node.neighbors:
            return None
        if np.random.rand() < 0.3:  # 10% chance of random exploration
            return np.random.choice(self.current_node.neighbors)
        # Choose the edge with the highest pheromone level
        chosen_edge = max(pheromone_levels, key=pheromone_levels.get)
        return chosen_edge
    
    def deposit_pheromones(self, edge):
        edge.pheromone_level[self.current_node] += 1
    
    def choose_edge(self):
        if not self.current_node.neighbors:
            return None
        if np.random.rand() < 0.3:  # 10% chance of random exploration
            return np.random.choice(self.current_node.neighbors)
        total_pheromone = sum(edge.pheromone_level[self.current_node] for edge in self.current_node.neighbors)
        probabilities = [edge.pheromone_level[self.current_node] / total_pheromone for edge in self.current_node.neighbors]
        chosen_edge = np.random.choice(self.current_node.neighbors, p=probabilities)
        return chosen_edge
    
    def move_towards(self, target_node):
        angle = math.atan2(target_node.y - self.current_node.y, target_node.x - self.current_node.x)
        self.current_node.x += math.cos(angle)
        self.current_node.y += math.sin(angle)
        # Check if the ant has reached the target node
        if math.hypot(self.current_node.x - target_node.x, self.current_node.y - target_node.y) < 1:
            self.current_node = target_node

    def move(self,ants):
    
        chosen_edge = self.choose_edge()
        if chosen_edge is None:
            return 
        next_node = chosen_edge.other_end(self.current_node)
        if not any(ant.current_node == next_node for ant in ants):  # Check if the next node is occupied
            self.move_towards(next_node)
        
            
        self.current_node = chosen_edge.other_end(self.current_node)
        if self.current_node.node_type == 'food':
            self.has_food = True
            self.current_node.food_amount -= 1
            if self.current_node.food_amount == 0:
                self.current_node.node_type ='empty'
        elif self.current_node.node_type == 'home':
            self.has_food = False
        pheromone_deposit = 3 if self.has_food else 1  # Deposit more pheromone when carrying food
        chosen_edge.pheromone_level[self.current_node] += pheromone_deposit


In [172]:
main()